In [61]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import pandas as pd
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, softmax
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

%matplotlib widget

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [62]:
df = pd.read_csv("../archive/song_data.csv")
df.drop_duplicates(subset='song_name', keep='first', inplace=True)
df.drop('loudness', axis=1)

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,1,0.0294,167.060,4,0.4740
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,0,0.0498,105.256,4,0.3700
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,1,0.0792,123.881,4,0.3240
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,1,0.1070,122.444,4,0.1980
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,1,0.0313,172.011,4,0.5740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18827,Stay Awake,55,114582,0.898000,0.370,0.136,0.000263,7,0.0999,1,0.0433,146.081,4,0.0592
18830,Let It Breathe,60,159645,0.893000,0.500,0.151,0.000065,11,0.1110,1,0.0348,113.969,4,0.3000
18832,Sudden Love (Acoustic),23,182211,0.847000,0.719,0.325,0.000000,0,0.1250,1,0.0355,130.534,4,0.2860
18833,Gentle on My Mind,55,352280,0.945000,0.488,0.326,0.015700,3,0.1190,1,0.0328,106.063,4,0.3230


In [63]:
X, y = df.drop('song_name',axis=1).to_numpy(), df['song_popularity'].to_numpy()

In [64]:
X

array([[7.30000e+01, 2.62333e+05, 5.52000e-03, ..., 1.67060e+02,
        4.00000e+00, 4.74000e-01],
       [6.60000e+01, 2.16933e+05, 1.03000e-02, ..., 1.05256e+02,
        4.00000e+00, 3.70000e-01],
       [7.60000e+01, 2.31733e+05, 8.17000e-03, ..., 1.23881e+02,
        4.00000e+00, 3.24000e-01],
       ...,
       [2.30000e+01, 1.82211e+05, 8.47000e-01, ..., 1.30534e+02,
        4.00000e+00, 2.86000e-01],
       [5.50000e+01, 3.52280e+05, 9.45000e-01, ..., 1.06063e+02,
        4.00000e+00, 3.23000e-01],
       [6.00000e+01, 1.93533e+05, 9.11000e-01, ..., 9.14900e+01,
        4.00000e+00, 5.81000e-01]])

In [65]:
""" from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor  # Assuming you're using a neural network for regression

# Define the pipeline with an imputer and your regression model (e.g., MLPRegressor)
my_regression_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),  # You can customize the imputer based on your needs
    ('model', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=0))  # Adjust hyperparameters
])

# Define the number of folds for cross-validation
k_folds = 5

# Initialize KFold
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Multiply by -1 since scikit-learn convention is to maximize scores, but you want to minimize MSE
mse_scores = -1 * cross_val_score(my_regression_pipeline, X, y, cv=kf, scoring='neg_mean_squared_error')

# Print MSE scores
print("MSE scores:\n", mse_scores)

# Calculate and print the average MSE score across experiments
print("Average MSE score (across experiments):")
print(np.mean(mse_scores)) """

' from sklearn.model_selection import cross_val_score, KFold\nfrom sklearn.impute import SimpleImputer\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.neural_network import MLPRegressor  # Assuming you\'re using a neural network for regression\n\n# Define the pipeline with an imputer and your regression model (e.g., MLPRegressor)\nmy_regression_pipeline = Pipeline(steps=[\n    (\'preprocessor\', SimpleImputer()),  # You can customize the imputer based on your needs\n    (\'model\', MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500, random_state=0))  # Adjust hyperparameters\n])\n\n# Define the number of folds for cross-validation\nk_folds = 5\n\n# Initialize KFold\nkf = KFold(n_splits=k_folds, shuffle=True, random_state=42)\n\n# Multiply by -1 since scikit-learn convention is to maximize scores, but you want to minimize MSE\nmse_scores = -1 * cross_val_score(my_regression_pipeline, X, y, cv=kf, scoring=\'neg_mean_squared_error\')\n\n# Print MSE scores\nprint("MSE scores:\n",

In [66]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


# Define the pipeline with an imputer and your regression model (MLPRegressor)
my_regression_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),  # You can customize the imputer based on your needs
    ('model', MLPRegressor(max_iter=500, random_state=0))  # Adjust other hyperparameters
])

# Define the hyperparameter grid to search
param_grid = {
    'model__hidden_layer_sizes': [(10,5), (100, 50), (15, 10, 5), (10,5,2), (12, 8, 4), (8, 4, 2)],
    'model__alpha': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'model__learning_rate_init': [0.001, 0.01, 0.1],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(my_regression_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Perform the hyperparameter search with cross-validation
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Emile Peeters\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Best Hyperparameters: {'model__alpha': 0.1, 'model__hidden_layer_sizes': (10, 5, 2), 'model__learning_rate_init': 0.001}


In [69]:
# Print mean test scores for each combination of hyperparameters
print("Mean Test Scores:")
print(grid_search.cv_results_['mean_test_score'])

# Print the index of the best hyperparameters in the results
best_index = grid_search.best_index_
print(f"\nBest Hyperparameters Index: {best_index}")

# Print the average performance of the best hyperparameters
print(f"\nAverage Performance of Best Hyperparameters: {grid_search.cv_results_['mean_test_score'][best_index]}")

Mean Test Scores:
[-2.61835564e+03 -9.20309744e+04 -2.44061531e+05 -4.56053478e+03
 -3.88904194e+03 -4.09293503e+02 -2.95493968e+02 -2.63877219e+03
 -4.09450259e+02 -1.53293539e+03 -2.33830756e+03 -4.09289837e+02
 -3.96445042e+03 -1.27114610e+03 -5.65202895e+02 -7.88809063e+02
 -4.09432329e+02 -4.09334704e+02 -2.48703595e+02 -4.09365546e+02
 -4.09441365e+02 -5.13306845e+02 -1.85211712e+04 -4.09404812e+02
 -3.39694900e+03 -2.67943263e+03 -1.86786860e+03 -4.09982267e+03
 -7.12272252e+04 -6.74221710e+02 -8.36925841e+02 -1.66529781e+03
 -4.09450258e+02 -5.97575471e+02 -8.48584994e+02 -4.09251727e+02
 -9.06584077e+04 -2.85395208e+03 -4.09280558e+02 -7.88217060e+00
 -6.05876957e+02 -4.09392930e+02 -1.00624182e+01 -4.09365804e+02
 -4.09405245e+02 -5.98143722e+02 -4.03280768e+03 -4.09377820e+02
 -7.89886532e+02 -1.20578515e+05 -2.23624022e+02 -1.19852239e+04
 -2.34115663e+04 -4.09276530e+02 -3.64424608e+03 -2.64217315e+03
 -4.09450225e+02 -5.91996201e+02 -4.09433230e+02 -4.09276080e+02
 -3.733

In [67]:
#!pip install scikeras[tensorflow]
#!pip install --upgrade tensorflow scikit-learn